# Redes Neuronales para credit scoring.

In [3]:
import pandas as pd
import numpy as np


# Carga de datos    
df = pd.read_csv('df_credit_scoring.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   checking_status      1000 non-null   object
 1   duration             1000 non-null   int64 
 2   credit_history       1000 non-null   object
 3   purpose              1000 non-null   object
 4   credit_amount        1000 non-null   int64 
 5   savings              1000 non-null   object
 6   employment           1000 non-null   object
 7   installment_rate     1000 non-null   int64 
 8   personal_status_sex  1000 non-null   object
 9   other_debtors        1000 non-null   object
 10  residence_since      1000 non-null   int64 
 11  property             1000 non-null   object
 12  age                  1000 non-null   int64 
 13  installment_plans    1000 non-null   object
 14  housing              1000 non-null   object
 15  existing_credits     1000 non-null   int64 
 16  job    

In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, Input

def create_1d_cnn(num_layers, filter_size, num_filters, input_shape, num_classes):
    model = Sequential()
    
    # Use Input layer as the first layer
    model.add(Input(shape=input_shape))

    # Add the first Conv1D layer
    model.add(Conv1D(filters=num_filters, kernel_size=filter_size, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    
    # Capas convolucionales adicionales, según se indique en num_layers
    for _ in range(num_layers - 1):
        model.add(Conv1D(filters=num_filters, kernel_size=filter_size, activation='relu'))
        model.add(MaxPooling1D(pool_size=2))
    
    # Capa de flatten y clasificación
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))  # Capa densa intermedia
    model.add(Dropout(0.5))  # Regularización para evitar sobreajuste
    model.add(Dense(num_classes, activation='softmax'))  # Capa de salida
    
    # Compilación del modelo
    model.compile(optimizer='adam', 
                  loss='sparse_categorical_crossentropy', 
                  metrics=['accuracy'])
    
    return model

# Ejemplo de uso
num_layers = 2        # Número de capas convolucionales
filter_size = 5        # Tamaño del filtro
num_filters = 32       # Número de filtros por capa
input_shape = (20, 1) # Longitud de secuencia de entrada y 1 canal
num_classes = 2       # Número de clases a predecir

# Crear el modelo
model = create_1d_cnn(num_layers, filter_size, num_filters, input_shape, num_classes)

# Mostrar la arquitectura del modelo
model.summary()

2024-10-15 08:00:10.147120: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-10-15 08:00:10.147182: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-10-15 08:00:10.147197: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-10-15 08:00:10.147435: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-10-15 08:00:10.147461: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 16, 32)         │           192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 8, 32)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 4, 32)          │         5,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 2, 32)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,634 (37.63 KB)

 Trainable params: 9,634 (37.63 KB)

 Non-trainable params: 0 (0.00 B)

In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RepeatedKFold, train_test_split
from sklearn.metrics import f1_score
import numpy as np


def prep_data(df):
    # Dividir las columnas numéricas y categóricas
    numerical_features = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
    categorical_features = df.select_dtypes(include=['object']).columns.tolist()
    # Definir el preprocesador para las columnas numéricas
    numerical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),  # Imputación con la media
        ('scaler', MinMaxScaler())                    # Escalado Min-Max
    ])

    # Definir el preprocesador para las columnas categóricas
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),  # Imputación con la moda
        ('onehot', OneHotEncoder(handle_unknown='ignore'))     # Codificación One-Hot
    ])

    # Combinar ambos preprocesadores en un ColumnTransformer
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, numerical_features),
            ('cat', categorical_transformer, categorical_features)
        ])

    return preprocessor.fit_transform(df), preprocessor


def repeated_kfold_cross_validation(params, df, target_column = 'class', n_splits=5, n_repeats=3, epochs=10, batch_size=32):
    """
    Realiza validación cruzada repetida utilizando RepeatedKFold.
    
    Parámetros:
    -----------
    - model: Modelo a evaluar.
    - df: DataFrame de pandas con las características y etiquetas.
    - target_column: Nombre de la columna que contiene las etiquetas.
    - n_splits: Número de folds en cada iteración de k-Fold.
    - n_repeats: Número de repeticiones de k-Fold.
    - epochs: Número de épocas para el entrenamiento.
    - batch_size: Tamaño del lote durante el entrenamiento.

    Retorna:
    --------
    - Lista con las precisiones obtenidas en cada fold de todas las repeticiones.
    """

    # Extraer las características (X) y etiquetas (y)
    X = df.drop(columns=[target_column])#.values
    y = df[target_column].values
    # Crear el objeto RepeatedKFold
    rkf = RepeatedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=42)

    all_scores = {'accuracy':[],
                  'f1_score':[]}

    # Iterar sobre los folds generados por RepeatedKFold
    for fold, (train_index, val_index) in enumerate(rkf.split(X)):
        print(f"Fold {fold + 1}/{n_splits * n_repeats}")

        # Preparar los datos del fold
        X_train, X_val = X.loc[train_index], X.loc[val_index]
        y_train, y_val = y[train_index], y[val_index]

        # transformar variables no numericas y valores nulos
        X_train, preprocessor = prep_data(X_train)
        X_val = preprocessor.transform(X_val)

        # Asegurar que los datos tengan la forma correcta: (n_samples, timesteps, channels)
        X_train = X_train[..., np.newaxis]
        X_val = X_val[..., np.newaxis]

        # Crear y compilar un nuevo modelo para este fold
        #num_layers = int(params[0])
        #filter_size = int(params[1])
        #num_filters = int(params[2])
        #batch_size = int(params[3])
        print(params)

        num_layers,filter_size, num_filters, batch_size = map(int, params.flatten())
        input_shape = (X_train.shape[1], 1) # Longitud de secuencia de entrada y 1 canal
        num_classes = 2       # Número de clases a predecir

        model = create_1d_cnn(num_layers, filter_size, num_filters, input_shape, num_classes)
        
        if model.optimizer is None:
            model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy','f1_score'])
        # Entrenar el modelo
        model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)

        # Evaluar el modelo en los datos de validación
        loss, accuracy = model.evaluate(X_val, y_val, verbose=0)
        print(f"  Accuracy: {accuracy:.4f}")
        all_scores['accuracy'].append(accuracy)
        
        y_pred = model.predict(X_val)
        y_pred_classes = np.argmax(y_pred, axis=1)

        f1 = f1_score(y_val, y_pred_classes)

        all_scores['f1_score'].append(f1)

    # Calcular y mostrar la precisión promedio
    mean_accuracy = np.mean(all_scores['accuracy'])
    print(f"\nPrecision promedio en {n_repeats} repeticiones de {n_splits}-Fold: {mean_accuracy:.4f}")

    return -np.mean(all_scores['f1_score'])

In [6]:
# Crear y compilar un nuevo modelo para este fold
num_layers = 2        # Número de capas convolucionales
filter_size = 5        # Tamaño del filtro
num_filters = 32       # Número de filtros por capa
input_shape = (61, 1) # Longitud de secuencia de entrada y 1 canal
num_classes = 2       # Número de clases a predecir


model = create_1d_cnn(num_layers, filter_size, num_filters, input_shape, num_classes)

In [7]:
X = df.drop(columns = 'class')
y = df['class']

X, preprocessor = prep_data(X)

In [8]:
X[..., np.newaxis].shape

(1000, 61, 1)

In [11]:
model.fit(tf.convert_to_tensor(X[..., np.newaxis]), y, verbose = 0);

In [49]:
params

[2, 5, 32, 16]

In [56]:
params = [2,
          5,
          32,
          16]

all_scores = repeated_kfold_cross_validation(params=np.array(params), df= df, target_column='class', n_splits=2, n_repeats=3, epochs=5, batch_size=32)

Fold 1/6
[ 2  5 32 16]


/opt/anaconda3/envs/si_proyecto/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


  Accuracy: 0.7060
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
Fold 2/6
[ 2  5 32 16]


/opt/anaconda3/envs/si_proyecto/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


  Accuracy: 0.6940
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
Fold 3/6
[ 2  5 32 16]


...

  Accuracy: 0.7080
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
Fold 5/6
[ 2  5 32 16]


/opt/anaconda3/envs/si_proyecto/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


  Accuracy: 0.6860
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
Fold 6/6
[ 2  5 32 16]


/opt/anaconda3/envs/si_proyecto/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


  Accuracy: 0.7140
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step

Precision promedio en 3 repeticiones de 2-Fold: 0.7000


In [57]:
all_scores

-0.82348924416047

In [12]:
from pyswarms.single.global_best import GlobalBestPSO

# num_layers,filter_size, num_filters, batch_size

lb = [1, 2, 16, 16]  # Límites inferiores (capas, filtros, tamaño del filtro, batch size)
ub = [2, 5, 64, 128]  # Límites superiores


def fitness_function(params):
    # 'params' is a 2D array: (n_particles, n_dimensions)
    results = []

    # Loop through each particle's parameters
    for particle in params:
        # Unpack parameters for each particle
        #num_layers, filter_size, num_filters, batch_size = map(int, particle)
        # Collect the negative accuracy (for minimization)
        results.append(repeated_kfold_cross_validation(particle, df= df))

    # Return the results as a numpy array
    return np.array(results)



def run_pso_optimization(df):
 
    # Definir función lambda para pasar parámetros adicionales a la función objetivo
    #fitness_function = lambda params: repeated_kfold_cross_validation(params, df= df)

    # Ejecutar PSO
    optimizer =  GlobalBestPSO(n_particles = 10, 
                               dimensions = 4,
                               options={'c1': 0.5, 'c2': 0.3, 'w': 0.9}, 
                               bounds= (lb, ub))
    
    best_params, best_score = optimizer.optimize(fitness_function, iters = 5)

    print(f'Mejores Hiperparámetros: {best_params}')
    print(f'Mejor Precisión: {-best_score:.4f}')



In [61]:
run_pso_optimization(df)

2024-10-14 21:50:15,443 - pyswarms.single.global_best - INFO - Optimize for 5 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best:   0%|          |0/5/opt/anaconda3/envs/si_proyecto/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Fold 1/15
[  1.91587057   4.13881122  63.82076395 121.21482119]
  Accuracy: 0.7050
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Fold 2/15
[  1.91587057   4.13881122  63.82076395 121.21482119]


/opt/anaconda3/envs/si_proyecto/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-10-14 21:50:20,818 - tensorflow - WARNING - 5 out of the last 24 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x31134f6a0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org

  Accuracy: 0.7050
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Fold 3/15


/opt/anaconda3/envs/si_proyecto/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

  Accuracy: 0.7250
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step
Fold 5/15
[ 1.83923112  2.88440621 39.82129304 94.55015432]


/opt/anaconda3/envs/si_proyecto/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


  Accuracy: 0.7000
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step
Fold 6/15
[ 1.83923112  2.88440621 39.82129304 94.55015432]


/opt/anaconda3/envs/si_proyecto/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
pyswarms.single.global_best:  80%|████████  |4/5, best_cost=-.823


KeyboardInterrupt: 

In [20]:
from tensorflow.keras import backend as K

def simplified_model_evaluation(params, df, target_column = 'class', n_splits=5, n_repeats=3, epochs=5, batch_size=32):
    # Extraer las características (X) y etiquetas (y)
    X = df.drop(columns=[target_column])#.values
    y = df[target_column].values
    # Crear el objeto RepeatedKFold
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2)

    X_train, preprocessor = prep_data(X_train)
    X_val = preprocessor.transform(X_val)

    # Asegurar que los datos tengan la forma correcta: (n_samples, timesteps, channels)
    X_train = tf.convert_to_tensor(X_train[..., np.newaxis])
    X_val = tf.convert_to_tensor(X_val[..., np.newaxis])

    num_layers,filter_size, num_filters, batch_size = map(int, params.flatten())
    input_shape = (X_train.shape[1], 1) # Longitud de secuencia de entrada y 1 canal
    num_classes = 2       # Número de clases a predecir

    model = create_1d_cnn(num_layers, filter_size, num_filters, input_shape, num_classes)
    
    if model.optimizer is None:
        model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
    # Entrenar el modelo
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0);
    
    y_pred = model.predict(X_val);
    y_pred_classes = np.argmax(y_pred, axis=1)

    f1 = f1_score(y_val, y_pred_classes)

    K.clear_session()

    return -f1

In [14]:
params = [2,
          5,
          32,
          16]

simplified_model_evaluation(np.array(params), df)

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step


-0.8130563798219584

In [21]:
from pyswarms.single.global_best import GlobalBestPSO

# num_layers,filter_size, num_filters, batch_size

lb = [1, 1, 16, 16]  # Límites inferiores (capas, filtros, tamaño del filtro, batch size)
ub = [2, 4, 64, 128]  # Límites superiores


def fitness_function(params):
    # 'params' is a 2D array: (n_particles, n_dimensions)
    results = []

    # Loop through each particle's parameters
    for particle in params:
        # Unpack parameters for each particle
        #num_layers, filter_size, num_filters, batch_size = map(int, particle)
        # Collect the negative accuracy (for minimization)
        results.append(simplified_model_evaluation(particle, df= df))

    # Return the results as a numpy array
    return np.array(results)



def run_pso_optimization(df, options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9}, n_particles = 2, iters = 2, verbose = True):
 
    # Definir función lambda para pasar parámetros adicionales a la función objetivo
    #fitness_function = lambda params: repeated_kfold_cross_validation(params, df= df)

    # Ejecutar PSO
    optimizer =  GlobalBestPSO(n_particles = n_particles, 
                               dimensions = 4,
                               options=options, 
                               bounds= (lb, ub))
    
    best_score, best_params = optimizer.optimize(fitness_function, iters = iters, verbose = verbose)

    if verbose:
        print(f'Mejores Hiperparámetros: {best_params}')
        print(f'Mejor f1: {-best_score}')
    
    return best_params, -best_score

def experimento_clase(df, swarms = 30, iters = 2, n_particles = 2):
    Ws = [0.9, 0.8, 0.7]
    c1s = [2, 1.5, 0.8]
    c2s = [2, 1.5, 0.8]

    metrics_dict = {'W':[],
                    'C1':[],
                    'C2':[],
                    'f1_score':[],
                    'best_params':[],
                    'swarm':[]}
    for w in Ws:
        for c1 in c1s:
            for c2 in c2s:
                for i in range(swarms):

                    print(f"    Realizando el enjambre {i+1}/{swarms} \n")

                    metrics_dict['W'].append(w)
                    metrics_dict['C1'].append(c1)
                    metrics_dict['C2'].append(c2)
                    metrics_dict['swarm'].append(i)

                    options = {'c1': c1, 'c2': c2, 'w': w}

                    best_params, f1 = run_pso_optimization(df, 
                                                           options=options, 
                                                           iters=iters,
                                                           n_particles=n_particles,
                                                           verbose=False)

                    best_params_str = f"num_layers:{int(best_params[0])}, filter_size:{int(best_params[1])}, num_filters:{int(best_params[2])}, batch_size:{int(best_params[3])}"

                    metrics_dict['best_params'].append(best_params_str)
                    metrics_dict['f1_score'].append(f1)
                
                print("     ---------- terminados los enjambres \n")

    return metrics_dict


In [11]:
run_pso_optimization(df)

2024-10-15 01:26:15,417 - pyswarms.single.global_best - INFO - Optimize for 2 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best:   0%|          |0/2/opt/anaconda3/envs/si_proyecto/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-10-15 01:26:15.915693: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


/opt/anaconda3/envs/si_proyecto/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step


pyswarms.single.global_best:  50%|█████     |1/2, best_cost=-.86/opt/anaconda3/envs/si_proyecto/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

2024-10-15 01:26:23,956 - tensorflow - WARNING - 5 out of the last 15 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x31b5a4900> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step


pyswarms.single.global_best: 100%|██████████|2/2, best_cost=-.86
2024-10-15 01:26:24,059 - pyswarms.single.global_best - INFO - Optimization finished | best cost: -0.8603988603988605, best pos: [ 1.76979485  2.50744158 46.85815699 74.62252454]


Mejores Hiperparámetros: [ 1.76979485  2.50744158 46.85815699 74.62252454]
Mejor f1: 0.8603988603988605


(array([ 1.76979485,  2.50744158, 46.85815699, 74.62252454]),
 0.8603988603988605)

In [12]:
metrics = experimento_clase(df, swarms=2)
metrics

2024-10-15 01:26:28,423 - pyswarms.single.global_best - INFO - Optimize for 2 iters with {'c1': 2, 'c2': 2, 'w': 0.9}


    Realizando el enjambre 1/2 



pyswarms.single.global_best:   0%|          |0/2/opt/anaconda3/envs/si_proyecto/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step


/opt/anaconda3/envs/si_proyecto/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

/opt/anaconda3/envs/si_proyecto/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


pyswarms.single.global_best: 100%|██████████|2/2, best_cost=-.827
2024-10-15 01:26:45,044 - pyswarms.single.global_best - INFO - Optimization finished | best cost: -0.8269794721407624, best pos: [ 1.74424055  2.30569658 40.19016153 90.53608846]


     ---------- terminados los enjambres 



{'W': [0.9, 0.9],
 'C1': [2, 2],
 'C2': [2, 2],
 'f1_score': [0.8439306358381503, 0.8269794721407624],
 'best_params': ['num_layers:1, filter_size:3, num_filters:23, batch_size:115',
  'num_layers:1, filter_size:2, num_filters:40, batch_size:90'],
 'swarm': [0, 1]}

In [13]:
pd.DataFrame(metrics)

,W,C1,C2,f1_score,best_params,swarm
0,0.9,2,2,0.843931,"num_layers:1, filter_size:3, num_filters:23, b...",0
1,0.9,2,2,0.826979,"num_layers:1, filter_size:2, num_filters:40, b...",1


In [25]:
import csv

def experimento_clase(df, swarms=2, iters=2, n_particles=2, output_file='metrics.csv'):
    Ws = [0.9, 0.8]#, 0.7]
    c1s = [2, 1.5]#, 0.8]
    c2s = [2, 1.5]#, 0.8]

    # Escribir el encabezado al inicio
    with open(output_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['W', 'C1', 'C2', 'f1_score', 'best_params', 'swarm'])

    # Iterar por cada combinación de hiperparámetros
    for w in Ws:
        for c1 in c1s:
            for c2 in c2s:
                for i in range(swarms):
                    print(f"    Realizando el enjambre {i+1}/{swarms} \n")

                    # Definir las opciones para el enjambre
                    options = {'c1': c1, 'c2': c2, 'w': w}

                    # Ejecutar la optimización PSO
                    best_params, f1 = run_pso_optimization(
                        df, options=options, iters=iters, n_particles=n_particles, verbose=False
                    )

                    # Formatear los mejores parámetros como cadena
                    best_params_str = (
                        f"num_layers:{int(best_params[0])}, "
                        f"filter_size:{int(best_params[1])}, "
                        f"num_filters:{int(best_params[2])}, "
                        f"batch_size:{int(best_params[3])}"
                    )

                    # Abrir el archivo para escribir una línea por vez
                    with open(output_file, mode='a', newline='') as file:
                        writer = csv.writer(file)
                        writer.writerow([w, c1, c2, f1, best_params_str, i])

                print("     ---------- Terminados los enjambres \n")

    print(f"Experimento completado. Resultados guardados en {output_file}")

In [23]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Disable TensorFlow INFO and WARNING logs

# Disable progress bars in TensorFlow
from tensorflow.keras.utils import disable_interactive_logging
disable_interactive_logging()


experimento_clase(df)

    Realizando el enjambre 1/2 

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step



7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
    Realizando el enjambre 2/2 

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
     ---------- Terminados los enjambres 

Experimento completado. Resultados guardados en metrics.csv


In [26]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Disable TensorFlow INFO and WARNING logs

# Disable progress bars in TensorFlow
from tensorflow.keras.utils import disable_interactive_logging
disable_interactive_logging()


experimento_clase(df, swarms=10, n_particles=6, iters=10)

    Realizando el enjambre 1/10 

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step



7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
7/7 ━━━━━━━━━━━━━━━

: 

: 

: 

In [24]:
import pandas as pd

metrics_df = pd.read_csv('metrics.csv')

metrics_df

,W,C1,C2,f1_score,best_params,swarm
0,0.9,2,2,0.870056,"num_layers:1, filter_size:2, num_filters:24, b...",0
1,0.9,2,2,0.860399,"num_layers:1, filter_size:1, num_filters:52, b...",1
